In [1]:
from utils import *
from atae_lstm import *
from torch.utils.data import DataLoader, random_split
import torch.nn as nn

In [2]:
TRAIN_FILE_PATH = '../data/absa/SemEval14/abas-pytorch/Laptops_Train.xml.seg'
TEST_FILE_PATH = '../data/absa/SemEval14/abas-pytorch/Laptops_Test_Gold.xml.seg'

In [3]:
BATCH_SIZE = 16
NUM_EPOCHS = 20

In [4]:
tokenizer = build_tokenizer(fnames=[TRAIN_FILE_PATH, TEST_FILE_PATH], 
                            max_seq_len=85,
                            dat_fname='./output/laptop_tokenizer.dat')

loading tokenizer:  ./output/laptop_tokenizer.dat
finish!


In [5]:
trainset = ABSADataset(TRAIN_FILE_PATH, tokenizer)
testset = ABSADataset(TEST_FILE_PATH, tokenizer)

print(len(trainset))
val_len = int(len(trainset) * 0.1)
trainset, valset = random_split(trainset, [len(trainset) - val_len, val_len])
print(len(trainset))

2328
2096


In [9]:
x1 = [1, 2]
x2 = [11, 22, 33, 44]
x_len = [2, 4]
x_len = torch.tensor(x_len)
x1_pad = pad_and_truncate(x1, 4)
x2_pad = pad_and_truncate(x2, 4)
print('x1: ', x1_pad)
print('x2: ', x2_pad)

x_batch = np.vstack((x1_pad, x2_pad))
x_batch = torch.tensor(x_batch)
print('x_batch: ', x_batch)

embedding = nn.Embedding(1000, 3)
x_batch_embd = embedding(x_batch)
print(x_batch_embd)

rnn_test = DynamicRNN(3, 6)
out, (ht, ct) = rnn_test(x_batch_embd, x_len)

x1:  [1 2 0 0]
x2:  [11 22 33 44]
x_batch:  tensor([[ 1,  2,  0,  0],
        [11, 22, 33, 44]])
tensor([[[-0.9197,  0.2485, -1.0594],
         [-0.7178, -1.2320, -0.9252],
         [ 0.8024,  0.1987, -0.0424],
         [ 0.8024,  0.1987, -0.0424]],

        [[-0.0301,  1.1027, -0.3387],
         [-1.6352, -0.1453, -0.1169],
         [-1.1660, -0.2237, -0.0090],
         [-1.0424, -1.9349, -1.3921]]], grad_fn=<EmbeddingBackward0>)
rnn forward....
x raw:  tensor([[[-0.9197,  0.2485, -1.0594],
         [-0.7178, -1.2320, -0.9252],
         [ 0.8024,  0.1987, -0.0424],
         [ 0.8024,  0.1987, -0.0424]],

        [[-0.0301,  1.1027, -0.3387],
         [-1.6352, -0.1453, -0.1169],
         [-1.1660, -0.2237, -0.0090],
         [-1.0424, -1.9349, -1.3921]]], grad_fn=<EmbeddingBackward0>)
x len:  tensor([2, 4])
x_sort_idx:  tensor([1, 0])
x_sort:  tensor([[[-0.0301,  1.1027, -0.3387],
         [-1.6352, -0.1453, -0.1169],
         [-1.1660, -0.2237, -0.0090],
         [-1.0424, -1.9349, -